In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gspread
%matplotlib inline

from collections import Counter

pd.options.display.max_columns = None

X:\Users\AC\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
url2 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
df_usconf = pd.read_csv(url,error_bad_lines = False)
df_usdead = pd.read_csv(url2,error_bad_lines = False)

df_counties = pd.read_csv(r'X:\AC\Documents\Datasets\US Census and OMB Data\2020 Counties UID State CBSA CSA.csv', delimiter = ',', encoding = "ISO-8859-1")
df_counties = df_counties[['UID', 'CBSA Code', 'CBSA Title', 'CSA Code', 'CSA Title']]

#Add CBSA/CSA titles and codes to df
df_usconf = df_usconf.merge(df_counties, on = 'UID', how = 'left', suffixes=(False,False))
df_usconf = df_usconf.merge(df_usdead[['UID','Population']], on = 'UID', how = 'left', suffixes=(False,False))
cols = df_usconf.columns.tolist()
df_usconf = df_usconf[cols[0:1] + cols[-1:] + cols[5:6] + cols[6:8] + cols[-5:-1] + cols[11:-5]]

#Check to ensure no missing UID or odd formatting changes
if df_usconf['UID'].isna().sum() > 0:
    print('Check for empty UIDs')
    
#replace NaNs with ''
df_usconf.fillna('', inplace=True)

In [3]:
df_usconf.head()

,UID,Population,Admin2,Province_State,Country_Region,CBSA Code,CBSA Title,CSA Code,CSA Title,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20
0,84001001,55869,Autauga,Alabama,US,33860,"Montgomery, AL",388,"Montgomery-Selma-Alexander City, AL",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,6,6,6,6,6,7,8,10,12,12,12,12,12,12,15,17,19,19,19,23,24,26,26,25,26,28,30,32,33,36,36,37,39,40,43,44,42,45,48,53,53,58,61,67,68,74,84,91,93,104,103,110,110,120,127,136,143,149,155,159,168,189,192,205,212,216,221,233,239,239,241,248,259,265,272,282,295,312,323,331,357,364,367,369,394,405,425,428,436,447,463,473,482,492,497,521,530,545,553,560,583,607,610,636,643,653,662,676,698,715,733,743,765,773,810,823,838,845,863,877,891,900,909,932,941,949,963,972,988,1010,1024,1030,1030,1050,1065,1086,1169,1174,1162,1188,1196,1196,1206,1218,1219,1235,1241,1240,1255,1264,1266,1286,1286,1281,1284,1296,1309,1345,1348,1354,1345,1349,1355,1371,1377,1383,1385,1398,1413,1420,1432,1442,1447,1463,1619,1624,1664,1673,1690,1691,1714,1715,1738,1757,1764,1773,1785,1787,1791,1798,1805,1818,1828,1831,1839,1852,1863,1882,1898,1905,1911,1924,1928,1949,1966,1983,1989,1999,2010,2021,2023,2030,2048,2059,2074,2082,2103,2126,2141,2159,2173
1,84001003,223234,Baldwin,Alabama,US,19300,"Daphne-Fairhope-Foley, AL",380,"Mobile-Daphne-Fairhope, AL",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,2,2,2,3,4,4,5,5,10,15,18,19,20,24,28,29,29,38,42,44,56,59,66,71,72,87,91,101,103,109,112,117,123,132,143,147,147,161,168,171,174,174,175,181,187,188,189,196,205,208,216,222,224,227,231,243,244,254,254,260,262,270,269,271,273,274,274,277,282,281,282,283,290,292,292,292,293,296,304,313,320,325,331,343,353,361

In [4]:
#Guam, American Samoa, Diamond/Grand Princess, Northern Mariana Islands, Virgin islands all have NaN in Admin2
#Removing nan in Admin2 column so that I can combine 'Out of' and 'Unassigned' Admin2
df_usconf.loc[df_usconf['Admin2'].isna(),'Admin2'] = ''

#loop to combine 'Out of *' and 'Unassigned' Admin2's
dates = df_usconf.columns[9:].tolist()
for state in df_usconf.loc[df_usconf['Admin2'].str.contains('Unassigned'),'Province_State'].unique():
    a = df_usconf.loc[(df_usconf['Admin2'].str.contains('Unassigned')) & (df_usconf['Province_State']==state)]
    b = df_usconf.loc[(df_usconf['Admin2'].str.contains('Out of')) & (df_usconf['Province_State']==state)]
    df_usconf.loc[(df_usconf['Admin2'].str.contains('Unassigned')) & (df_usconf['Province_State']==state), dates]= a.iloc[:,9:].values + b.iloc[:,9:].values
    df_usconf = df_usconf.drop(b.index)

In [5]:
#convert date columns into a single column for cumulative totals df
df_usconf_pivot = df_usconf.melt(id_vars = df_usconf.columns[:9], var_name = 'Date', value_name = 'Total Confirmed Cases').sort_values(by = ['UID','Date']).reset_index(drop=True)
df_usdead_pivot = df_usdead.melt(id_vars = df_usdead.columns[:12], var_name = 'Date', value_name = 'Total Dead').sort_values(by = ['UID','Date']).reset_index(drop=True)

#create df for daily increments
df_usconf_daily = df_usconf.copy()
df_usdead_daily = df_usdead.copy()
df_usconf_daily.iloc[:,9:] = df_usconf_daily.iloc[:,9:].diff(axis=1).fillna(0).astype('int')
df_usdead_daily.iloc[:,12:] = df_usdead_daily.iloc[:,12:].diff(axis=1).fillna(0).astype('int')

#convert date columns into a single column for daily increments
df_usconf_daily = df_usconf_daily.melt(id_vars = df_usconf_daily.columns[:9], var_name = 'Date', value_name = 'Daily Confirmed Cases').sort_values(by = ['UID','Date']).reset_index(drop=True)
df_usdead_daily = df_usdead_daily.melt(id_vars = df_usdead_daily.columns[:12], var_name = 'Date', value_name = 'Daily Dead').sort_values(by = ['UID','Date']).reset_index(drop=True)

In [6]:
#combine cumulative totals for deaths and confirmed cases into one df
us_totals = df_usconf_pivot.merge(df_usdead_pivot[['UID','Date','Total Dead']], on = ['UID','Date'], how = 'left', suffixes = (False, False))
us_totals['Date'] = pd.to_datetime(us_totals['Date']).apply(lambda x: pd.datetime.strftime(x, '%m/%d/%y'))
us_totals = us_totals.sort_values(by = ['UID','Date']).reset_index(drop=True)

#combine daily increase for deaths and confirmed cases into one df
us_daily = df_usconf_daily.merge(df_usdead_daily[['UID','Date','Daily Dead']], on = ['UID','Date'], how = 'left', suffixes = (False, False))
us_daily['Date'] = pd.to_datetime(us_daily['Date']).apply(lambda x: pd.datetime.strftime(x, '%m/%d/%y'))
us_daily = us_daily.sort_values(by = ['UID','Date']).reset_index(drop=True)

X:\Users\AC\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  This is separate from the ipykernel package so we can avoid doing imports until
X:\Users\AC\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


In [7]:
mid = int(len(us_daily)/2)
us_daily[mid:mid + 10]

,UID,Population,Admin2,Province_State,Country_Region,CBSA Code,CBSA Title,CSA Code,CSA Title,Date,Daily Confirmed Cases,Daily Dead
468540,84029157,19136,Perry,Missouri,US,,,,,01/22/20,0,0
468541,84029157,19136,Perry,Missouri,US,,,,,01/23/20,0,0
468542,84029157,19136,Perry,Missouri,US,,,,,01/24/20,0,0
468543,84029157,19136,Perry,Missouri,US,,,,,01/25/20,0,0
468544,84029157,19136,Perry,Missouri,US,,,,,01/26/20,0,0
468545,84029157,19136,Perry,Missouri,US,,,,,01/27/20,0,0
468546,84029157,19136,Perry,Missouri,US,,,,,01/28/20,0,0
468547,84029157,19136,Perry,Missouri,US,,,,,01/29/20,0,0
468548,84029157,19136,Perry,Missouri,US,,,,,01/30/20,0,0
468549,84029157,19136,Perry,Missouri,US,,,,,01/31/20,0,0


In [8]:
us_daily.loc[(us_daily['Date']==max(us_daily['Date'])) & (~us_daily['Admin2'].isin(['Unassigned','']))].tail()

,UID,Population,Admin2,Province_State,Country_Region,CBSA Code,CBSA Title,CSA Code,CSA Title,Date,Daily Confirmed Cases,Daily Dead
920834,84070016,0,Central Utah,Utah,US,,,,,11/01/20,47,0
921119,84070017,0,Southeast Utah,Utah,US,,,,,11/01/20,18,1
921404,84070018,0,Southwest Utah,Utah,US,,,,,11/01/20,94,0
921689,84070019,0,TriCounty,Utah,US,,,,,11/01/20,16,0
921974,84070020,0,Weber-Morgan,Utah,US,,,,,11/01/20,188,0


In [9]:
#Daily update for spreadsheet 4
#Date range is ordered from oldest to newest
#client email: host-937@tableau-automation-286223.iam.gserviceaccount.com
gc = gspread.service_account(filename='./covid-19-data/client_secret.json')
sheet4 = gc.open("COVID Tableau Automation 4").sheet1

current = list(set(sheet4.col_values(10)))

if len(current) == 0:
    sheet4.append_rows([us_daily.columns.values.tolist()])

need_to_update= []
for date in us_daily['Date'].unique()[230:].tolist():
    if date in (us_daily['Date'].unique()[230:305]) and date not in current:
        need_to_update.append(date)
    if date not in (us_daily['Date'].unique()[230:305]):
        print('You reached the spreadsheet limit. \nStart next spreadsheet at: ',date)
        break

if len(need_to_update) == 0:
    print('Spreadsheet already updated')
else:
    us_daily_4 = us_daily.copy().loc[us_daily['Date'].isin(need_to_update)]
    sheet4.append_rows(us_daily_4.values.tolist())
    print('Dates added to google sheet 4: {}'.format(need_to_update))

Dates added to google sheet 4: ['11/01/20']


In [10]:
statedaily = us_daily.loc[~us_daily['Province_State'].isin(['Diamond Princess','Grand Princess'])].groupby(['Province_State','Date'], as_index=False)['Population','Daily Confirmed Cases'].sum()
statedaily = statedaily.sort_values(['Date','Province_State'], ascending = [False,True]).reset_index(drop=True)
statecumsum = us_totals.groupby(['Province_State','Date'], as_index=False)['Total Confirmed Cases','Total Dead'].sum()
stategrp = pd.merge(statedaily, statecumsum, on = ['Province_State','Date'], how = 'left', suffixes = (False, False))

X:\Users\AC\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.
X:\Users\AC\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
conf_total = us_daily['Daily Confirmed Cases'].sum()
conf_dead = us_daily['Daily Dead'].sum()
US_pop = 329943320 #as of 1/1/20


print('US Stats As of: ', us_totals['Date'].sort_values(ascending = True).to_list()[-1])
print('Total Confirmed Cases To Date: ', conf_total)
print('Confirmed Cases Percentage of US population: %.2f' %((conf_total/ US_pop) * 100),'%')
print('\n')
print('Total Deaths To Date: ', (conf_dead))
print('Confirmed Cases Percentage of US population: %.2f' %((conf_dead/ US_pop) * 100),'%')
print('Percentage of deaths from confirmed cases : %.2f' %((conf_dead/ US_pop)/(conf_total/ US_pop)*100),'%')
print('\n')

US Stats As of:  11/01/20
Total Confirmed Cases To Date:  9206974
Confirmed Cases Percentage of US population: 2.79 %


Total Deaths To Date:  230766
Confirmed Cases Percentage of US population: 0.07 %
Percentage of deaths from confirmed cases : 2.51 %




In [13]:
last14 = us_daily['Date'].values.tolist()[-14:]
prev2weekavg = us_daily.loc[(us_daily['Date'].isin(us_daily['Date'].values.tolist()[-15:-1]))].groupby('Date')['Daily Confirmed Cases'].sum().mean()
yestsum = us_daily.loc[(us_daily['Date']==us_daily['Date'].unique()[-1])]['Daily Confirmed Cases'].sum()

print('For Yesterday ({}):'.format(us_daily['Date'].unique()[-1]))
print('Increase in Total Confirmed Cases: ', yestsum)
print('Percentage increase from average of last two weeks: ', ((yestsum - prev2weekavg)/prev2weekavg * 100), '%')
print('\n')
print('Last 14 days:')
print('Total Confirmed Cases: ', us_daily.loc[(us_daily['Date'].isin(last14))]['Daily Confirmed Cases'].sum())
print('Average Cases per day: %.1f' %prev2weekavg)
print('Cases in Last 14 days as Percentage of Total Cases: %.2f' %((us_daily.loc[(us_daily['Date'].isin(last14))]['Daily Confirmed Cases'].sum()/(conf_total))*100),'%')

For Yesterday (10/20/20):
Increase in Total Confirmed Cases:  60315
Percentage increase from average of last two weeks:  11.75667931480361 %


Last 14 days:
Total Confirmed Cases:  773955
Average Cases per day: 53969.9
Cases in Last 14 days as Percentage of Total Cases: 9.35 %


In [11]:
#Feature creation, rankings
stategrp['Total Cases per 1000 capita'] = stategrp['Total Confirmed Cases']/stategrp['Population']*1000

ranks = []
for date in stategrp['Date'].unique():
    for ranking in stategrp.copy().loc[stategrp['Date']==date,'Total Confirmed Cases'].rank(ascending = False, method='min'):
        ranks.append(ranking)
rankings = pd.DataFrame({'Total Cases Daily Ranking': ranks})
stategrp = pd.concat([stategrp, rankings], axis=1)

dates = stategrp['Date'].unique().tolist()
ustemp = pd.DataFrame()
for state in stategrp['Province_State'].unique():
    statetemp = stategrp.loc[stategrp['Province_State']==state].copy().reset_index(drop=True)
    yest = pd.Series(statetemp.loc[statetemp['Date'].isin(dates[1:]),'Total Cases Daily Ranking'].reset_index(drop=True)).rename('Total Cases Ranking Daily Change')
    statetemp = pd.concat([statetemp,yest], axis=1)
    ustemp = pd.concat([ustemp, statetemp])

stategrp = ustemp.sort_values(['Date','Total Cases Daily Ranking'], ascending = [False,True]).reset_index(drop=True)
stategrp['Total Cases Ranking Daily Change'] = stategrp['Total Cases Ranking Daily Change'] - stategrp['Total Cases Daily Ranking']

ranks = []
for date in stategrp['Date'].unique():
    for ranking in stategrp.copy().loc[stategrp['Date']==date,'Daily Confirmed Cases'].rank(ascending = False, method='min'):
        ranks.append(ranking)
rankings = pd.DataFrame({'Daily Cases Ranking': ranks})
stategrp = pd.concat([stategrp, rankings], axis=1)

ranks = []
for date in stategrp['Date'].unique():
    for ranking in stategrp.copy().loc[stategrp['Date']==date,'Total Cases per 1000 capita'].rank(ascending = False, method='min'):
        ranks.append(ranking)
rankings = pd.DataFrame({'Cases per Capita Ranking': ranks})
stategrp = pd.concat([stategrp, rankings], axis=1)

In [ ]:
us_daily.to_csv(r'X:\AC\Documents\Datasets\US_daily_pivot.csv', index=False)
us_totals.to_csv(r'X:\AC\Documents\Datasets\US_totals_pivot.csv', index=False)

In [ ]:
#create extract for google sheets
#consider removing 'Northern Mariana Islands, Guam, American Samoa, Diamond Princess, Grand Princess Virgin Islands, Virginn Islands' to save space
#will need to split into multiple sheets
#consolidate the Admin2s: 'Out of *, Unassigned' They seem to be staging values, all balance out to 0

# Data Exploration

In [ ]:
stategrp[0:9]

In [ ]:
#Top daily increases by state
stategrp.sort_values(by='Daily Confirmed Cases', ascending = False)[0:9]

In [ ]:
#top daily increases of cases per capita
stategrp.assign(x = stategrp['Daily Confirmed Cases']/stategrp['Population']).sort_values(by='x', ascending = False).drop('x',axis=1)[0:9]

In [ ]:
#Top daily increases by county
us_daily.sort_values(by=['Daily Confirmed Cases'], ascending = False)[['Province_State','Admin2','Date','Population','Daily Confirmed Cases','Daily Dead']][0:5]

# CA Data Exploration

In [ ]:
us_daily.loc[(us_daily['Province_State']=='California')&(us_daily['Admin2']=='San Francisco'),['Date','Daily Confirmed Cases']].tail()

In [ ]:
#Top daily increases for CA
stategrp.loc[stategrp['Province_State']=='California'].sort_values(by=['Daily Confirmed Cases'], ascending = False)[0:4]

In [ ]:
#Create df for CA's CBSAs
cacbsa_daily = us_daily.loc[(us_daily['Province_State']=='California')].groupby(['Province_State','CBSA Title','Date'], as_index=False)['Population','Daily Confirmed Cases'].sum()
cacbsa_daily = cacbsa_daily.sort_values(['Date','Daily Confirmed Cases'], ascending = [False,False]).reset_index(drop=True)
cacbsa_cumsum = us_totals.loc[us_totals['Province_State']=='California'].groupby(['Province_State','CBSA Title','Date'], as_index=False)['Total Confirmed Cases','Total Dead'].sum()
cacbsa = pd.merge(cacbsa_daily, cacbsa_cumsum, on = ['Province_State','CBSA Title','Date'], how = 'left', suffixes = (False, False))

In [ ]:
#Feature creation, rankings
cacbsa['Total Cases per 1000 capita'] = cacbsa['Total Confirmed Cases']/cacbsa['Population']*1000

ranks = []
for date in cacbsa['Date'].unique():
    for ranking in cacbsa.copy().loc[cacbsa['Date']==date,'Total Confirmed Cases'].rank(ascending = False, method='min'):
        ranks.append(ranking)
rankings = pd.DataFrame({'Total Cases Daily Ranking': ranks})
cacbsa = pd.concat([cacbsa, rankings], axis=1)

dates = cacbsa['Date'].unique().tolist()
cacbsatemp = pd.DataFrame()
for state in cacbsa['CBSA Title'].unique():
    cbsatemp = cacbsa.loc[cacbsa['CBSA Title']==state].copy().reset_index(drop=True)
    yest = pd.Series(cbsatemp.loc[cbsatemp['Date'].isin(dates[1:]),'Total Cases Daily Ranking'].reset_index(drop=True)).rename('Total Cases Ranking Daily Change')
    cbsatemp = pd.concat([cbsatemp,yest], axis=1)
    cacbsatemp = pd.concat([cacbsatemp, cbsatemp])
    
cacbsa = cacbsatemp.sort_values(['Date','Total Cases Daily Ranking'], ascending = [False,True]).reset_index(drop=True)
cacbsa['Total Cases Ranking Daily Change'] = cacbsa['Total Cases Ranking Daily Change'] - cacbsa['Total Cases Daily Ranking']

In [ ]:
cacbsa.sort_values('Daily Confirmed Cases', ascending = False)

In [ ]:
cacbsa.loc[cacbsa['Date']==cacbsa['Date'][0]][0:5]

In [ ]:
#Most recent day in SF Oakland Berkeley CBSA
us_daily.loc[(us_daily['CBSA Title']=='San Francisco-Oakland-Berkeley, CA')&(us_daily['Date']==us_daily['Date'].max())][['CBSA Title','Admin2','Date','Population','Daily Confirmed Cases']]

In [ ]:
#county totals summary for CA
us_daily.loc[us_daily['Province_State']=='California'].groupby(['Admin2'])[['Population','Daily Confirmed Cases', 'Daily Dead']].agg({'Population':'mean','Daily Confirmed Cases':'sum', 'Daily Dead':'sum'}).sort_values('Daily Confirmed Cases', ascending = False)[0:4]

In [ ]:
us_daily['Date'].unique()[-30:]

In [ ]:
sf_lastmonth = us_daily.loc[(us_daily['Admin2']=='San Francisco')&(us_daily['Date'].isin(us_daily['Date'].unique()[-30:]))]

In [ ]:
#3208 cases in SF county in the last month
sf_lastmonth['Daily Confirmed Cases'].sum()

In [ ]:
#US census estimate for SF county population in 2019
sf_lastmonth['Population'].values[0]

In [ ]:
# .4 cases per 100 capita. Less than one in 100 are actively transmissible
sf_lastmonth['Daily Confirmed Cases'].sum()/sf_lastmonth['Population'].values[0]*100

In [ ]:
# .82 cases per 100 capita. Less than one in 100 have tested positive
us_daily.loc[(us_daily['Admin2']=='San Francisco')]['Daily Confirmed Cases'].sum()/sf_lastmonth['Population'].values[0]*100

In [ ]:
us_daily.loc[(us_daily['Admin2']=='San Francisco')]['Daily Confirmed Cases'].sum()/(sf_lastmonth['Population'].values[0]-us_daily.loc[(us_daily['Admin2']=='San Francisco')]['Daily Confirmed Cases'].sum())

In [ ]:
0.008266986610180929 * 100 * 25

In [ ]:
100 - (20000/(330000000-20000))*100

# APPENDIX

In [3]:
#temporarily removing the bottom two rows. Someone decided to reformat the source data...
# df_usconf = df_usconf.loc[~df_usconf['UID'].isna()]

In [6]:
# Need to review all features required for viz
# Combined observations (out of state, unassigned)
# replace np.nan with ''
# Load all values into gspread
# How to incrementally update instead of updating everything

#3288 different state/Admin2 groupings
#125-126 Dates can fit in one google spreadsheet
#Update seems limited to ~55 dates (2.2M cells), even though it's well bellow then 5M cell limit/sheet
    #this was due to the other columns that were prepopulated. Delete these prior to update

#tableau desktop can only export google sheets less than 10MB
#4.9M cells = 15.9MB excel file. Try 2/3 upload
#75-80 dates can be exported (use 75 to be safe)

In [82]:
#Initial Load
gc = gspread.service_account(filename='./covid-19-data/client_secret.json')
sheet = gc.open("COVID Tableau Automation 1").sheet1

#clear existing data
sheet.clear()

#delete extraneous columns to make room for more rows:
sheet.delete_columns(13,26)

us_daily_1 = us_daily.copy().loc[us_daily['Date'].isin(us_daily['Date'].unique()[:80])]
sheet.update([us_daily_1.columns.values.tolist()] + us_daily_1.values.tolist())

#2nd spreadsheet
sheet2 = gc.open("COVID Tableau Automation 2").sheet1

sheet2.delete_columns(13,26)

us_daily_2 = us_daily.copy().loc[us_daily['Date'].isin(us_daily['Date'].unique()[80:155])]
sheet2.update([us_daily_2.columns.values.tolist()] + us_daily_2.values.tolist())

{'spreadsheetId': '1kQEyp1E5gDs6DE3WPn8w71zxAWqpXH7qHKkcfKlsvtk',
 'updatedRange': 'Sheet1!A1:L411001',
 'updatedRows': 411001,
 'updatedColumns': 12,
 'updatedCells': 4932012}

In [6]:
#Daily update for spreadsheet 3
#Date range is ordered from oldest to newest
#client email: host-937@tableau-automation-286223.iam.gserviceaccount.com
gc = gspread.service_account(filename='./covid-19-data/client_secret.json')
sheet3 = gc.open("COVID Tableau Automation 3").sheet1

current = list(set(sheet3.col_values(10)))
need_to_update= []
for date in us_daily['Date'].unique()[155:].tolist():
    if date in (us_daily['Date'].unique()[155:230]) and date not in current:
        need_to_update.append(date)
    if date not in (us_daily['Date'].unique()[155:230]):
        print('You reached the spreadsheet limit. \nStart next spreadsheet at: ',date)
        break

if len(need_to_update) == 0:
    print('Spreadsheet already updated')
else:
    us_daily_3 = us_daily.copy().loc[us_daily['Date'].isin(need_to_update)]
    sheet3.append_rows(us_daily_3.values.tolist())
    print('Dates added to google sheet 3: {}'.format(need_to_update))

You reached the spreadsheet limit. 
Start next spreadsheet at:  09/08/20
Spreadsheet already updated


In [ ]:
#create loop to create new spreadsheets and load data
#Not sure where gc.create created the new spreadsheet
# gc.create('COVID Tableau Automation {}'.format('3'))
gc = gspread.service_account(filename='./covid-19-data/client_secret.json')
for number in range(0,int(round(us_daily['Date'].unique().shape[0] / 75))):
#     Create new spreadsheet
#     Delete columns
#     create df
#     update with df
    

In [ ]:
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file('./covid-19-data/client_secret.json')

In [ ]:
url3 = 'https://covidtracking.com/api/v1/states/daily.csv'
df_testing = pd.read_csv(url3,error_bad_lines = False)

df_testing = df_testing.rename(columns = {'date':'Date','state':'Province_State'})
df_testing[['totalTestResultsIncrease','positiveIncrease','negativeIncrease','totalTestResults','positive','negative']] = df_testing[['totalTestResultsIncrease','positiveIncrease','negativeIncrease','totalTestResults','positive','negative']].fillna(0)

to_fix = ['totalTestResultsIncrease','positiveIncrease','negativeIncrease']
actual = ['totalTestResults','positive','negative']
columns = df_testing.columns
def fill_func(states):
    for state in states:
        for col in range(0,len(to_fix)):
            cumsum = df_testing.loc[df_testing['Province_State']==state].sort_values(by='Date')[to_fix[col]].cumsum()
            actualsum = df_testing.loc[df_testing['Province_State']==state,['Date',actual[col]]].sort_values(by='Date')[actual[col]]
            comparison = cumsum.eq(actualsum)
            if comparison[comparison == False].count() > 0:
                Earliest_index = comparison[comparison == False].index[0]
                df_testing.iloc[Earliest_index,columns.get_loc(to_fix[col])] = df_testing.iloc[Earliest_index,columns.get_loc(actual[col])]
                
fill_func(df_testing['Province_State'].unique())
df_testing = df_testing.replace({'Province_State':states})
df_testing['Date'] = pd.to_datetime(df_testing['Date'], format='%Y%m%d').apply(lambda x: pd.datetime.strftime(x, '%m/%d/%y'))



In [ ]:
#missing CBSA codes after merge: 41980, 10380, 38660, 11640, 41900, 49500, 32420, 25020, 27580, 17620, 17640, 42180
#They're all in Puerto Rico. Span across multiple CBSA and CSA. Should just ignore
res = Counter(df_counties['CBSA Code'].value_counts().to_dict()) - Counter(df_usconf['CBSA Code'].value_counts().to_dict())
pd.set_option('display.max_rows', None)
df_counties[df_counties['CBSA Code'].isin(list(res.keys()))]

In [ ]:
df_usconf.loc[df_usconf['UID']==630,['CBSA Code', 'CBSA Title', 'CSA Code', 'CSA Title']] = [41980,'San Juan-Bayamón-Caguas, PR', 490.0, 'San Juan-Bayamón, PR']
df_usconf.loc[df_usconf['UID']==630][['CBSA Code', 'CBSA Title', 'CSA Code', 'CSA Title']]

In [ ]:
columns_to_remove = ['iso2', 'iso3', 'code3', 'Combined_Key']
df_usconf = df_usconf[Counter(df_usconf.columns.tolist()) - Counter(columns_to_remove)]
df_usdead = df_usdead[Counter(df_usdead.columns.tolist()) - Counter(columns_to_remove)]

In [ ]:
#Add additional column for CBSAs and CSAs before melting?

'''BA_counties = ['Alameda','Contra Costa','Marin','Napa','San Francisco','San Mateo','Santa Clara','Solano','Sonoma']
LA_counties =['Ventura','San Bernadio', 'Riverside', 'Los Angeles', 'Orange']'''

In [ ]:
#California and Texas total confirmed cases differ by 2000+. Stick to COVIDTESTING data for testing dataframe
totalgrp = us_totals.groupby(['Province_State','Date'], as_index=False)['Total Confirmed Cases','Total Dead'].sum()
comparison = df_testing.loc[df_testing['Date']=='07/27/20', ['Province_State','Date','positive']].merge(totalgrp.loc[totalgrp['Date']=='07/27/20',['Province_State','Date','Total Confirmed Cases']], on=['Province_State','Date'], how = 'left', suffixes = (False, False))
comparison['Delta'] = comparison['positive'] - comparison['Total Confirmed Cases']
comparison